In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# Profiling and Tracing

* [Instance-based Profiling](inst_profile.ipynb)
* [Memory Profiling](inst_mem_profile.ipynb)
* [Instance-based Call Tracing](inst_call_tracing.ipynb)

The profiling and call tracing tools mentioned above have a similar programmatic interface, even though most of the time they will only be used in command-line mode. However, if you really want to customize the set of methods that are to be profiled or traced, see the following example.

``` python
    # for performance profiling
    from openmdao.devtools import iprofile as tool

    # OR for call tracing
    # from openmdao.devtools import itrace as tool

    # First, make sure that the classes I use to define my method set exist in this
    # namespace.
    from mystuff import MyClass, MyOtherClass

    # Let's say I only want to track methods with 'foo' or 'bar' in the name that belong to
    # MyClass and ones with 'baz' in the name that belong to either MyClass or MyOtherClass.
    # I use the following glob patterns and tuples of classes to specify this.
    methods = [
        ('*foo*', (MyClass,)),
        ('*bar*', (MyClass,)),
        ('*baz*', (MyClass, MyOtherClass))
    ]

    # set up the tool using my custom method set
    tool.setup(methods=methods)

    tool.start()

    # run the code I want to profile/trace...

    tool.stop()

    # do some other stuff that I don't want to profile/trace...
```

* [Timing Systems under MPI](timing.ipynb)

## Dumping debug info to file based on MPI rank and/or PID

The `OPENMDAO_DUMP` environment variable can be set to a comma separated list of keywords to control 
what information is dumped and which file it is written to. Some valid example vaues are 
`stdout`, `stderr`, `rank`, `pid`, `rank,pid`, `pid,rank`, `rank,trace`, or `pid,trace,args`.

- `rank` means to include the rank in the dump file name, e.g., om_dump_0.out
- `pid` means to include the pid in the dump file name, e.g., om_dump_12345.out
- `stdout` means to dump to stdout (so rank and pid are ignored)
- `stderr` means to dump to stderr (so rank and pid are ignored)
- `trace` means to show function entry and exit.
- `args` means to include args and kwargs passed to functions if `trace` is active.

if `rank` and `pid` are both included, the file name will include both, `rank` first, e.g., om_dump_0_12345.out

If `OPENMDAO_DUMP` is just a plain truthy value, like '1', then output is dumped to a file named om_dump.out.

The output written to the `om_dump` file will be the output of any `om_dump` calls that have been 
added to the code for debugging purposes, plus function entries and exits if `trace` is active.

The specific behavior of the `trace` option can be specified by calling the `set_trace_predicate`
method, which can be found in `openmdao.utils.general_utils`.  It takes a single argument which is
a function of the form `myfunc(name, obj)`, where `name` is the name of the object `obj` in its parent
container.  Note that `obj` is guaranteed to be a function.